# Model Training

In [23]:
# Setting up saving model path
import os
import sys

folder_to_save = 'trained_models/first_pass'

## Logistic Regression

In [5]:
import os
import numpy as np
import cv2
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# Load images and create raw pixel embeddings
def load_images_and_labels(image_dir):
    images = []
    labels = []
    label_map = {
        'Monkeypox': 0,
        'Chickenpox': 1,
        'Measles': 2,
        'Normal': 3
    }

    for class_name, label in label_map.items():
        class_dir = os.path.join(image_dir, class_name)
        for image_file in os.listdir(class_dir):
            img_path = os.path.join(class_dir, image_file)
            img = cv2.imread(img_path)
            img = cv2.resize(img, (224, 224))  # Resize to 224x224
            img = img.flatten()  # Flatten the image to raw pixel embedding
            images.append(img)
            labels.append(label)
    
    return np.array(images), np.array(labels)

# Load images from the dataset
image_dir = 'data'  # Base directory
X, y = load_images_and_labels(image_dir)

# First, split the dataset into 85% train+validation and 15% test sets
X_train_val, X_test, y_train_val, y_test = train_test_split(X, y, test_size=0.15, random_state=42, stratify=y)

# Next, split the train+validation set into 70% train and 15% validation
X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=0.1765, random_state=42, stratify=y_train_val)

# Train the logistic regression model with early stopping based on validation accuracy
def train_logistic_regression_with_early_stopping(X_train, y_train, X_val, y_val, max_epochs=100, patience=5):
    best_model = None
    best_acc = 0.0
    epochs_no_improve = 0

    for epoch in range(max_epochs):
        print(f'Epoch {epoch+1}/{max_epochs}')
        print('-' * 10)

        # Train Logistic Regression model
        model = LogisticRegression(max_iter=10000, solver='lbfgs', multi_class='multinomial')
        model.fit(X_train, y_train)

        # Validation phase
        val_preds = model.predict(X_val)
        val_acc = accuracy_score(y_val, val_preds)
        print(f'Validation Accuracy: {val_acc:.4f}')

        # Check if this is the best accuracy we've seen
        if val_acc > best_acc:
            best_acc = val_acc
            best_model = model  # Save the best model
            epochs_no_improve = 0  # Reset the counter if validation improves
            print(f"Validation accuracy improved to {val_acc:.4f}, saving model...")
        else:
            epochs_no_improve += 1
            print(f"No improvement. Epochs without improvement: {epochs_no_improve}")

        # Early stopping condition
        if epochs_no_improve >= patience:
            print(f"Early stopping triggered after {patience} epochs of no improvement.")
            break

    return best_model, best_acc

# Train the logistic regression model with early stopping
logistic_regression_best_model, best_val_acc = train_logistic_regression_with_early_stopping(X_train, y_train, X_val, y_val, max_epochs=100, patience=5)

# Evaluate the model on the test set
def evaluate_logistic_regression(model, X_test, y_test):
    test_preds = model.predict(X_test)

    # Calculate accuracy
    test_acc = accuracy_score(y_test, test_preds)
    print(f'Test Accuracy: {test_acc:.4f}')

    # Classification report and confusion matrix
    print("\nClassification Report:\n", classification_report(y_test, test_preds, target_names=['Monkeypox', 'Chickenpox', 'Measles', 'Normal']))
    print("\nConfusion Matrix:\n", confusion_matrix(y_test, test_preds))

# Evaluate the best model on the test set
evaluate_logistic_regression(logistic_regression_best_model, X_test, y_test)


c:\Users\Lucas\Documents\GitHub\skin-condition-classification\.venv\Lib\site-packages\sklearn\linear_model\_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(


Accuracy: 0.5584

Classification Report:
               precision    recall  f1-score   support

           0       0.57      0.48      0.52        56
           1       0.39      0.52      0.45        21
           2       0.35      0.39      0.37        18
           3       0.69      0.69      0.69        59

    accuracy                           0.56       154
   macro avg       0.50      0.52      0.51       154
weighted avg       0.57      0.56      0.56       154


Confusion Matrix:
 [[27 10  6 13]
 [ 4 11  4  2]
 [ 4  4  7  3]
 [12  3  3 41]]


In [11]:
# Export the model
import joblib
model_file = f'{folder_to_save}/logistic_regression.pkl'

joblib.dump(logistic_regression_best_model, model_file)
print(f"Model saved to {model_file}")

Model saved to model.pkl


## SVM for Classification

In [14]:
import os
import numpy as np
import cv2
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# Load images and create raw pixel embeddings
def load_images_and_labels(image_dir):
    images = []
    labels = []
    label_map = {
        'Monkeypox': 0,
        'Chickenpox': 1,
        'Measles': 2,
        'Normal': 3
    }

    for class_name, label in label_map.items():
        class_dir = os.path.join(image_dir, class_name)
        for image_file in os.listdir(class_dir):
            img_path = os.path.join(class_dir, image_file)
            img = cv2.imread(img_path)
            img = cv2.resize(img, (224, 224))  # Resize to 224x224
            img = img.flatten()  # Flatten the image to raw pixel embedding
            images.append(img)
            labels.append(label)
    
    return np.array(images), np.array(labels)

# Load images from the dataset
image_dir = 'data'  # Base directory
X, y = load_images_and_labels(image_dir)

# First, split the dataset into 85% train+validation and 15% test sets
X_train_val, X_test, y_train_val, y_test = train_test_split(X, y, test_size=0.15, random_state=42, stratify=y)

# Next, split the train+validation set into 70% train and 15% validation
X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=0.1765, random_state=42, stratify=y_train_val)

# Train the SVM model with early stopping based on validation accuracy
def train_svm_with_early_stopping(X_train, y_train, X_val, y_val, max_epochs=100, patience=5):
    best_model = None
    best_acc = 0.0
    epochs_no_improve = 0

    for epoch in range(max_epochs):
        print(f'Epoch {epoch+1}/{max_epochs}')
        print('-' * 10)

        # Train the SVM model
        model = SVC(kernel='linear', C=1, decision_function_shape='ovr')  # Linear kernel
        model.fit(X_train, y_train)

        # Validation phase
        val_preds = model.predict(X_val)
        val_acc = accuracy_score(y_val, val_preds)
        print(f'Validation Accuracy: {val_acc:.4f}')

        # Check if this is the best accuracy we've seen
        if val_acc > best_acc:
            best_acc = val_acc
            best_model = model  # Save the best model
            epochs_no_improve = 0  # Reset the counter if validation improves
            print(f"Validation accuracy improved to {val_acc:.4f}, saving model...")
        else:
            epochs_no_improve += 1
            print(f"No improvement. Epochs without improvement: {epochs_no_improve}")

        # Early stopping condition
        if epochs_no_improve >= patience:
            print(f"Early stopping triggered after {patience} epochs of no improvement.")
            break

    return best_model, best_acc

# Train the SVM model with early stopping
best_svm_model, best_val_acc = train_svm_with_early_stopping(X_train, y_train, X_val, y_val, max_epochs=100, patience=5)

# Evaluate the model on the test set
def evaluate_svm(model, X_test, y_test):
    test_preds = model.predict(X_test)

    # Calculate accuracy
    test_acc = accuracy_score(y_test, test_preds)
    print(f'Test Accuracy: {test_acc:.4f}')

    # Classification report and confusion matrix
    print("\nClassification Report:\n", classification_report(y_test, test_preds, target_names=['Monkeypox', 'Chickenpox', 'Measles', 'Normal']))
    print("\nConfusion Matrix:\n", confusion_matrix(y_test, test_preds))

# Evaluate the best model on the test set
evaluate_svm(best_svm_model, X_test, y_test)


Accuracy: 0.5649

Classification Report:
               precision    recall  f1-score   support

           0       0.60      0.59      0.59        56
           1       0.35      0.43      0.38        21
           2       0.33      0.39      0.36        18
           3       0.73      0.64      0.68        59

    accuracy                           0.56       154
   macro avg       0.50      0.51      0.51       154
weighted avg       0.58      0.56      0.57       154


Confusion Matrix:
 [[33  9  6  8]
 [ 5  9  4  3]
 [ 4  4  7  3]
 [13  4  4 38]]


In [15]:
# Export the model
import joblib
model_file = f'{folder_to_save}/svm.pkl'

joblib.dump(best_svm_model, model_file)
print(f"Model saved to {model_file}")

Model saved to trained_models/first_pass/svm_model.pkl


In [ ]:
# Export the model
import joblib
model_file = 'trained_models/first_pass/best_svm_model.pkl'

joblib.dump(best_svm_model, model_file)
print(f"Model saved to {model_file}")

# ResNet50

In [22]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np
import cv2
import os
from torch.utils.data import DataLoader, TensorDataset
from torchvision import models, transforms

# Load images and create raw pixel embeddings
def load_images_and_labels(image_dir):
    images = []
    labels = []
    label_map = {
        'Monkeypox': 0,
        'Chickenpox': 1,
        'Measles': 2,
        'Normal': 3
    }

    for class_name, label in label_map.items():
        class_dir = os.path.join(image_dir, class_name)
        for image_file in os.listdir(class_dir):
            img_path = os.path.join(class_dir, image_file)
            img = cv2.imread(img_path)
            img = cv2.resize(img, (224, 224))  # Resize to 224x224 for ResNet
            img = np.transpose(img, (2, 0, 1))  # Convert to CxHxW format
            images.append(img)
            labels.append(label)
    
    return np.array(images), np.array(labels)

# Load images from the dataset
image_dir = 'data'  # Base directory
X, y = load_images_and_labels(image_dir)

# First, split the dataset into 85% train+validation and 15% test sets
X_train_val, X_test, y_train_val, y_test = train_test_split(X, y, test_size=0.15, random_state=42, stratify=y)

# Next, split the train+validation set into 70% train and 15% validation
X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=0.1765, random_state=42, stratify=y_train_val)

# Convert the numpy arrays to PyTorch tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.long)
X_val_tensor = torch.tensor(X_val, dtype=torch.float32)
y_val_tensor = torch.tensor(y_val, dtype=torch.long)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.long)

# Create PyTorch datasets and dataloaders
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
val_dataset = TensorDataset(X_val_tensor, y_val_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Define a ResNet model (ResNet50) and modify the final layer for 4-class classification
model = models.resnet50(pretrained=True)

# Replace the final fully connected layer (ResNet50 has 2048 input features to the final layer)
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 4)  # 4 classes (Monkeypox, Chickenpox, Measles, Normal)

# Move the model to GPU if available
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model = model.to(device)

# Define loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Function to train the model with early stopping based on validation accuracy
def train_model_with_early_stopping(model, criterion, optimizer, num_epochs=10, patience=3):
    best_model_wts = None
    best_acc = 0.0
    epochs_no_improve = 0
    stop_training = False

    for epoch in range(num_epochs):
        if stop_training:
            break

        print(f'Epoch {epoch+1}/{num_epochs}')
        print('-' * 10)

        # Training phase
        model.train()
        running_loss = 0.0
        running_corrects = 0

        for inputs, labels in train_loader:
            inputs = inputs.to(device)
            labels = labels.to(device)

            optimizer.zero_grad()

            # Forward pass
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            loss = criterion(outputs, labels)

            # Backward pass and optimize
            loss.backward()
            optimizer.step()

            # Track loss and accuracy
            running_loss += loss.item() * inputs.size(0)
            running_corrects += torch.sum(preds == labels.data)

        epoch_loss = running_loss / len(train_loader.dataset)
        epoch_acc = running_corrects.double() / len(train_loader.dataset)

        print(f'Training Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')

        # Validation phase
        model.eval()
        val_corrects = 0
        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs = inputs.to(device)
                labels = labels.to(device)

                outputs = model(inputs)
                _, preds = torch.max(outputs, 1)

                val_corrects += torch.sum(preds == labels.data)

        val_acc = val_corrects.double() / len(val_loader.dataset)
        print(f'Validation Accuracy: {val_acc:.4f}')

        # Check if this is the best accuracy we've seen
        if val_acc > best_acc:
            best_acc = val_acc
            best_model_wts = model.state_dict()  # Save the best model weights
            epochs_no_improve = 0  # Reset the counter if validation improves
            print(f"Validation accuracy improved to {val_acc:.4f}, saving model weights...")
        else:
            epochs_no_improve += 1
            print(f"No improvement. Epochs without improvement: {epochs_no_improve}")

        # Early stopping condition
        if epochs_no_improve >= patience:
            print(f"Early stopping triggered after {patience} epochs of no improvement.")
            stop_training = True

    # Load best model weights
    if best_model_wts is not None:
        model.load_state_dict(best_model_wts)
        print("Loaded best model weights.")

    return model, best_acc

# Train the model with early stopping
trained_model, best_val_acc = train_model_with_early_stopping(model, criterion, optimizer, num_epochs=10, patience=3)

# Function to evaluate the model on the test set
def evaluate_model(model, dataloader):
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    model.eval()  # Set the model to evaluation mode

    all_preds = []
    all_labels = []
    running_corrects = 0

    with torch.no_grad():
        for inputs, labels in dataloader:
            inputs = inputs.to(device)
            labels = labels.to(device)

            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)

            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

            running_corrects += torch.sum(preds == labels.data)

    # Calculate accuracy
    accuracy = running_corrects.double() / len(dataloader.dataset)
    print(f'Test Accuracy: {accuracy:.4f}')

    # Classification report and confusion matrix
    print("\nClassification Report:\n", classification_report(all_labels, all_preds, target_names=['Monkeypox', 'Chickenpox', 'Measles', 'Normal']))
    print("\nConfusion Matrix:\n", confusion_matrix(all_labels, all_preds))

# Evaluate the model on the test set
evaluate_model(trained_model, test_loader)


c:\Users\Lucas\Documents\GitHub\skin-condition-classification\.venv\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\Lucas\Documents\GitHub\skin-condition-classification\.venv\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to C:\Users\Lucas/.cache\torch\hub\checkpoints\resnet50-0676ba61.pth
100.0%


Epoch 1/10
----------
Training Loss: 1.0011 Acc: 0.6338
Validation Accuracy: 0.4655
Validation accuracy improved to 0.4655, saving model weights...
Epoch 2/10
----------


KeyboardInterrupt: 

In [ ]:
# Save the best model
model_file = f'{folder_to_save}/early_stopping_resnet50.pth'
torch.save(trained_model.state_dict(), model_file)
print(f"Best model saved to {model_file} with validation accuracy: {best_val_acc:.4f}")

# Vision Transformer

In [20]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import numpy as np
import cv2
import os
from torch.utils.data import DataLoader, TensorDataset
import timm  # For loading the pre-trained ViT model

# Load images and create raw pixel embeddings
def load_images_and_labels(image_dir):
    images = []
    labels = []
    label_map = {
        'Monkeypox': 0,
        'Chickenpox': 1,
        'Measles': 2,
        'Normal': 3
    }

    for class_name, label in label_map.items():
        class_dir = os.path.join(image_dir, class_name)
        for image_file in os.listdir(class_dir):
            img_path = os.path.join(class_dir, image_file)
            img = cv2.imread(img_path)
            img = cv2.resize(img, (224, 224))  # Resize to 224x224 for ViT
            img = np.transpose(img, (2, 0, 1))  # Convert to CxHxW format (ViT expects this)
            images.append(img)
            labels.append(label)
    
    return np.array(images), np.array(labels)

# Load images from the dataset
image_dir = 'data'  # Base directory
X, y = load_images_and_labels(image_dir)

# First, split the dataset into 85% train+validation and 15% test sets
X_train_val, X_test, y_train_val, y_test = train_test_split(X, y, test_size=0.15, random_state=42, stratify=y)

# Next, split the train+validation set into 70% train and 15% validation
X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=0.1765, random_state=42, stratify=y_train_val)

# Convert the numpy arrays to PyTorch tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.long)
X_val_tensor = torch.tensor(X_val, dtype=torch.float32)
y_val_tensor = torch.tensor(y_val, dtype=torch.long)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.long)

# Create PyTorch datasets and dataloaders
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
val_dataset = TensorDataset(X_val_tensor, y_val_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Load a pre-trained Vision Transformer (ViT) model from the timm library
model = timm.create_model('vit_base_patch16_224', pretrained=True, num_classes=4)  # 4 classes

# Move model to GPU if available
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model = model.to(device)

# Define loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Function to train the model with early stopping based on validation accuracy
def train_model_with_early_stopping(model, criterion, optimizer, num_epochs=10, patience=3):
    best_model_wts = None
    best_acc = 0.0
    epochs_no_improve = 0
    stop_training = False

    for epoch in range(num_epochs):
        if stop_training:
            break

        print(f'Epoch {epoch+1}/{num_epochs}')
        print('-' * 10)

        # Training phase
        model.train()
        running_loss = 0.0
        running_corrects = 0

        for inputs, labels in train_loader:
            inputs = inputs.to(device)
            labels = labels.to(device)

            optimizer.zero_grad()

            # Forward pass
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            loss = criterion(outputs, labels)

            # Backward pass and optimize
            loss.backward()
            optimizer.step()

            # Track loss and accuracy
            running_loss += loss.item() * inputs.size(0)
            running_corrects += torch.sum(preds == labels.data)

        epoch_loss = running_loss / len(train_loader.dataset)
        epoch_acc = running_corrects.double() / len(train_loader.dataset)

        print(f'Training Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')

        # Validation phase
        model.eval()
        val_corrects = 0
        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs = inputs.to(device)
                labels = labels.to(device)

                outputs = model(inputs)
                _, preds = torch.max(outputs, 1)

                val_corrects += torch.sum(preds == labels.data)

        val_acc = val_corrects.double() / len(val_loader.dataset)
        print(f'Validation Accuracy: {val_acc:.4f}')

        # Check if this is the best accuracy we've seen
        if val_acc > best_acc:
            best_acc = val_acc
            best_model_wts = model.state_dict()  # Save the best model weights
            epochs_no_improve = 0  # Reset the counter if validation improves
            print(f"Validation accuracy improved to {val_acc:.4f}, saving model weights...")
        else:
            epochs_no_improve += 1
            print(f"No improvement. Epochs without improvement: {epochs_no_improve}")

        # Early stopping condition
        if epochs_no_improve >= patience:
            print(f"Early stopping triggered after {patience} epochs of no improvement.")
            stop_training = True

    # Load best model weights
    if best_model_wts is not None:
        model.load_state_dict(best_model_wts)
        print("Loaded best model weights.")

    return model, best_acc

# Train the model with early stopping
trained_model, best_val_acc = train_model_with_early_stopping(model, criterion, optimizer, num_epochs=10, patience=3)

# Function to evaluate the model on the test set
def evaluate_model(model, dataloader):
    model.eval()  # Set the model to evaluation mode
    all_preds = []
    all_labels = []
    running_corrects = 0

    with torch.no_grad():
        for inputs, labels in dataloader:
            inputs = inputs.to(device)
            labels = labels.to(device)

            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)

            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

            running_corrects += torch.sum(preds == labels.data)

    # Calculate accuracy
    accuracy = running_corrects.double() / len(dataloader.dataset)
    print(f'Test Accuracy: {accuracy:.4f}')

    # Classification report and confusion matrix
    print("\nClassification Report:\n", classification_report(all_labels, all_preds, target_names=['Monkeypox', 'Chickenpox', 'Measles', 'Normal']))
    print("\nConfusion Matrix:\n", confusion_matrix(all_labels, all_preds))

# Evaluate the model on the test set
evaluate_model(trained_model, test_loader)


c:\Users\Lucas\Documents\GitHub\skin-condition-classification\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\Lucas\Documents\GitHub\skin-condition-classification\.venv\Lib\site-packages\huggingface_hub\file_download.py:139: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Lucas\.cache\huggingface\hub\models--timm--vit_base_patch16_224.augreg2_in21k_ft_in1k. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need 

Epoch 1/10
----------


KeyboardInterrupt: 

In [ ]:
# Save the best model
model_file = f'{folder_to_save}/early_stopping_vit.pth'
torch.save(trained_model.state_dict(), model_file)
print(f"Best model saved to {model_file} with validation accuracy: {best_val_acc:.4f}")